In [98]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [2]:
import tensorflow as tf

C:\Users\y63qiu\AppData\Local\Continuum\anaconda3\envs\mlbook\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [3]:
x = tf.Variable(3,name="x")
y = tf.Variable(4,name="y")
f = x*x*y +y +2

In [4]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [5]:
sess = tf.Session()

In [6]:
sess.run(x.initializer)

In [7]:
sess.run(y.initializer)

In [8]:
result = sess.run(f)

In [9]:
print(result)

42


In [10]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [11]:
print(result)

42


In [12]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

In [13]:
print(result)

42


In [14]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

42


In [15]:
sess.close()

# Managing Graphs

In [16]:
x1 = tf.Variable(1)

In [17]:
x1.graph is tf.get_default_graph()

True

In [18]:
graph = tf.Graph()

In [19]:
with graph.as_default():
    x2 = tf.Variable(2)

In [20]:
x2.graph is graph

True

In [21]:
x2.graph is tf.get_default_graph()

False

In [22]:
x1.graph is tf.get_default_graph()

True

# Lifecycle of a Node Value

In [23]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

In [24]:
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [25]:
with tf.Session() as sess:
    y_val,z_val = sess.run([y,z])
    print(y.eval())
    print(z.eval())

10
15


# Linear Regression with TensorFlow 

In [26]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [27]:
housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

In [28]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32,name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

In [29]:
with tf.Session() as sess:
    theta_value = theta.eval()

In [30]:
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

# Implementing Gradient Descent 

## Normalize the inputs

In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m,1)),scaled_housing_data]

## Manually Computing the Gradients

In [32]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0,1.0,seed=42),name="theta")
y_pred = tf.matmul(X,theta,name="predictions")
error = y_pred - y 
mse = tf.reduce_mean(tf.square(error),name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X),error)
training_op = tf.assign(theta, theta - learning_rate*gradients)

In [33]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch",epoch, "MSE = ", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
        

Epoch 0 MSE =  2.7544262
Epoch 100 MSE =  0.632222
Epoch 200 MSE =  0.5727805
Epoch 300 MSE =  0.5585007
Epoch 400 MSE =  0.54907
Epoch 500 MSE =  0.542288
Epoch 600 MSE =  0.53737885
Epoch 700 MSE =  0.533822
Epoch 800 MSE =  0.5312425
Epoch 900 MSE =  0.5293705


## Using autooff 

In [34]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0,1.0,seed=42),name="theta")
y_pred = tf.matmul(X,theta,name="predictions")
error = y_pred - y 
mse = tf.reduce_mean(tf.square(error),name="mse")
gradients = tf.gradients(mse,[theta])[0]
training_op = tf.assign(theta, theta - learning_rate*gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch",epoch, "MSE = ", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE =  2.7544262
Epoch 100 MSE =  0.632222
Epoch 200 MSE =  0.5727805
Epoch 300 MSE =  0.5585007
Epoch 400 MSE =  0.54907
Epoch 500 MSE =  0.54228795
Epoch 600 MSE =  0.5373789
Epoch 700 MSE =  0.533822
Epoch 800 MSE =  0.5312425
Epoch 900 MSE =  0.5293704


## Using an Optimizer

In [36]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0,1.0,seed=42),name="theta")
y_pred = tf.matmul(X,theta,name="predictions")
error = y_pred - y 
mse = tf.reduce_mean(tf.square(error),name="mse")
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch",epoch, "MSE = ", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
    
print("best_theta")

Epoch 0 MSE =  2.7544262
Epoch 100 MSE =  2.7544262
Epoch 200 MSE =  2.7544262
Epoch 300 MSE =  2.7544262
Epoch 400 MSE =  2.7544262
Epoch 500 MSE =  2.7544262
Epoch 600 MSE =  2.7544262
Epoch 700 MSE =  2.7544262
Epoch 800 MSE =  2.7544262
Epoch 900 MSE =  2.7544262
best_theta


# Saving and Restoring Models 

In [47]:
n_epochs = 1000

learning_rate = 0.1

x = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32,name="X")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")
y_pred = tf.matmul(X,theta,name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            print("Epoch",epoch,"MSE = ",mse.eval())
            save_path = saver.save(sess,"/Mysaver/my_model.ckpt")
            
        sess.run(training_op)
        
    best_theta = theta.eval()
    save_path = saver.save(sess,"/Mysaver/my_model_final.ckpt")
    

Epoch 0 MSE =  2.7544262
Epoch 100 MSE =  0.5279283
Epoch 200 MSE =  0.5244984
Epoch 300 MSE =  0.52433455
Epoch 400 MSE =  0.5243225
Epoch 500 MSE =  0.52432096
Epoch 600 MSE =  0.52432114
Epoch 700 MSE =  0.5243209
Epoch 800 MSE =  0.524321
Epoch 900 MSE =  0.5243211


In [48]:
save_path

'/Mysaver/my_model_final.ckpt'

In [51]:
# Restoring a model
with tf.Session() as sess:
    saver.restore(sess, "/Mysaver/my_model_final.ckpt")
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /Mysaver/my_model_final.ckpt


In [52]:
np.allclose(best_theta,best_theta_restored)

True

# Visualizing Graph

## create a time-stamp log directory 

In [56]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,now)

## Evaluate MSE and write to logfiles 

In [76]:
# Construction phase
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None,n + 1),name="X")
y = tf.placeholder(tf.float32, shape=(None,1),name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")
y_pred = tf.matmul(X,theta,name="predicitions")
error = y_pred-y
mse = tf.reduce_mean(tf.square(error),name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

# write MSE summary into a log file
mse_summary = tf.summary.scalar('MSE',mse)
file_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())

In [77]:
# mini-batch
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1,1)[indices]
    return X_batch, y_batch

In [79]:
# update execution phase to evaluate the mse_summary node
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch,batch_index,batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X:X_batch,y:y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
            
    best_theta = theta.eval()

In [82]:
file_writer.close()

In [70]:
mse

<tf.Tensor 'mse_12:0' shape=() dtype=float32>

In [92]:
from qyy import show_graph

# Name Scopes 

In [99]:
reset_graph()

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_dir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None,n+1),name="X")
y = tf.placeholder(tf.float32, shape=(None,1),name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")
y_pred = tf.matmul(X,theta,name="predictions")

In [100]:
# define the error and mse with a name scope called "loss"

with tf.name_scope("loss") as scope:
    error = y_pred-y
    mse = tf.reduce_mean(tf.square(error), name="mse")

In [96]:
print(error.op.name)

loss/sub


In [97]:
print(mse.op.name)

loss/mse


In [101]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE',mse)
file_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())

In [102]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch,y_batch = fetch_batch(epoch,batch_index,batch_size)
            if batch_index%10 == 0:
                summary_str = mse_summary.eval(feed_dict={X:X_batch, y:y_batch})
                
    best_theta = theta.eval()
    
file_writer.flush()
file_writer.close()


# Modularity

In [109]:
reset_graph()

n_features = 3
X = tf.placeholder(tf.float32,shape=(None,n_features),name="X")

w1 = tf.Variable(tf.random_normal((n_features,1)),name="weights1")
w2 = tf.Variable(tf.random_normal((n_features,1)),name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X,w1), b1, name="z1")
z2 = tf.add(tf.matmul(X,w2), b2, name="z2")

relu1 = tf.maximum(z1,0.,name="relu1")
relu2 = tf.maximum(z2,0.,name="relu2")

output = tf.add(relu1,relu2,name="output")

Much Better, using a function to bulid the ReLUs:

In [110]:
reset_graph()

In [118]:
def relu(X):
    w_shape = (int(X.get_shape()[1]),1)
    w = tf.Variable(tf.random_normal(w_shape),name="weights")
    b = tf.Variable(0.0,name="bias")
    z = tf.add(tf.matmul(X,w),b,name="z")
    return tf.maximum(z,0.,name="relu")

In [119]:
n_features = 3
X = tf.placeholder(tf.float32,shape=(None,n_features),name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus,name="output")

In [120]:
file_writer = tf.summary.FileWriter("logs/relu1",tf.get_default_graph())

Even better using name scopes


In [121]:
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]),1)
        w = tf.Variable(tf.random_normal(w_shape),name="weights")
        b = tf.Variable(0.0,name="bias")
        z = tf.add(tf.matmul(X,w),b,name="z")
        return tf.maximum(z,0.,name="max") 

In [124]:
n_features = 3
X = tf.placeholder(tf.float32,shape=(None,n_features),name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu2",tf.get_default_graph())
file_writer.close

<bound method FileWriter.close of <tensorflow.python.summary.writer.writer.FileWriter object at 0x000000001636B1D0>>

# Sharing Variable 